# [Macrophage and neutrophil heterogeneity at single-cell spatial resolution in human inflammatory bowel disease](https://pubmed.ncbi.nlm.nih.gov/37495570/)
GEO: [GSE214695](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE214695)

In [ ]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(dplyr)
    library(tibble)
})

In [ ]:
source('scTools/R/process.geo.data.r')

In [ ]:
seurat_objs <- process.GEO.data(GEOID = "GSE214695")

In [ ]:
# Merge samples
obj <- merge(seurat_objs[[1]], 
c(seurat_objs[[2]], seurat_objs[[3]], seurat_objs[[4]], seurat_objs[[5]], seurat_objs[[6]], seurat_objs[[7]],
 seurat_objs[[8]], seurat_objs[[9]], seurat_objs[[10]], seurat_objs[[11]], seurat_objs[[12]], seurat_objs[[13]],
seurat_objs[[14]], seurat_objs[[15]], seurat_objs[[16]], seurat_objs[[17]], seurat_objs[[18]]))
save(obj, file = file.path("objects", "00.rda"))

In [ ]:
# Clean up data
obj <- JoinLayers(obj)
obj[["percent.mt"]] <- PercentageFeatureSet(object = obj, pattern = "^MT-")
obj <- subset(obj, subset = nFeature_RNA > 350 & nFeature_RNA < 5000 & percent.mt < 10)
obj
save(obj, file = file.path("objects", "01.rda"))

In [ ]:
# length(rownames(annotations))
# length(rownames(df1_filtered))
# length(rownames(obj@meta.data))

In [ ]:
# common_row_names <- intersect(rownames(obj@meta.data), rownames(annotations))
# common_row_names
# df1_filtered <- obj@meta.data[common_row_names, ]

In [ ]:
obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = TRUE)
save(obj, file=file.path("objects", "02.rda"))

In [ ]:
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)
save(obj, file=file.path("objects", "03.rda"))

In [ ]:
# # Add updated metadata
# metadata <- obj@meta.data
# metadata <- left_join(
#   obj@meta.data %>% rownames_to_column(),
#   annotations %>% rownames_to_column(),
#   by = "rowname"
# )
# rownames(metadata) <- metadata$rowname
# metadata <- metadata[, -1]  # Remove the redundant rowname column
# obj@meta.data <- metadata
# obj

In [ ]:
# load(file = file.path("objects", "01.rda"))

In [ ]:
test <- obj

In [ ]:
rownames(test@meta.data) <- make.unique(gsub("[^A-Za-z]+", "", rownames(test@meta.data)))
# head(test@meta.data[1:4])

In [ ]:
mdata <- read.csv("GSE214695/GSE214695_cell_annotation.csv.gz", sep = ",")
mdata$cell_id <- make.unique(mdata$cell_id)
rownames(mdata) <- mdata$cell_id
rownames(mdata) <- make.unique(gsub("[^A-Za-z]+", "", rownames(mdata)))
# mdata <- select(mdata, c(sample, annotation, nanostring_reference))
# head(mdata)

In [ ]:
metadata <- merge(test@meta.data, mdata, by=0, all.x=TRUE)
# rownames(metadata) <- metadata$Row.names
metadata<- select(metadata, c(Row.names, sample, annotation, nanostring_reference))
# head(metadata)
# length(rownames(metadata))

In [ ]:
test <- AddMetaData(test, metadata = metadata, col.name = "sample")
test <- AddMetaData(test, metadata = metadata, col.name = "annotation")
test <- AddMetaData(test, metadata = metadata, col.name = "nanostring_reference")

In [ ]:
rownames(test@assays$RNA@cells) <- make.unique(gsub("[^A-Za-z]+", "", rownames(test@assays$RNA@cells)))

In [ ]:
Idents(test) <- "RNA"
test <- SCTransform(test, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = TRUE)
save(test, file=file.path("objects", "02.rda"))

In [ ]:
# load(file = file.path("objects", "02.rda"))

In [ ]:
test <- RunPCA(test, npcs = 30, verbose = TRUE)
test <- RunUMAP(test, reduction = "pca", dims = 1:20)
test <- FindNeighbors(test, reduction = "pca", dims = 1:20)
test <- FindClusters(test, resolution = 0.5)
save(test, file=file.path("objects", "03.rda"))

In [ ]:
immune_cell_names <- c(
  'CD8 CTL', 'Tregs', 'M2', 'PC IgA IgM', 'PC IgA heat shock 1', 'PC IgA 1',
  'PC IgA heat shock 2', 'ThF', 'PC IgA 3', 'Memory B cell', 'PC IgA Lambda 1',
  'CD4 ANXA1', 'NK', 'PC IgA 2', 'Cycling T cells', 'PC IgA 4', 'CD4 naïve',
  'CD8 CTL TRM', 'S1PR1 T cells', 'Plasmablast IgA Lambda 2', 'ILC3', 'GC B cell',
  'B cell', 'T cells CCL20', 'IDA macrophage', 'M1 ACOD1', 'PC IgG 1',
  'CD8 FGFBP2', 'DCs CD1c', 'M0_Ribhi', 'M1 CXCL5', 'Ribhi T cells 1',
  'Activated endothelium', 'M0', 'Ribhi T cells 2', 'DCs CCL22', 'B cell Ribhi',
  'M1 CXCL5', 'Tuft cells', 'Eosinophils', 'Cycling cells 3', 'Myofibroblasts',
  'Naïve B cell', 'DCs CCL22_Ribhi'
)

In [ ]:
load(file = file.path("objects", "03.rda"))

In [ ]:
test <- SetIdent(test, value = "annotation")
test <- subset(test, idents = immune_cell_names, invert = TRUE)

In [ ]:
blueprint.ref <- celldex::BlueprintEncodeData()
monaco.ref <- celldex::MonacoImmuneData()

In [ ]:
sce <- LayerData(test)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
test@meta.data$blueprint.main <- blueprint.main$pruned.labels
save(test, file=file.path("objects", "04.rda"))

In [ ]:
table(test@meta.data$blueprint.main)

In [ ]:
test <- SetIdent(test, value = "blueprint.main")
remove_low_count_cells <- function(seurat_obj, metadata_column, threshold = 20) {

  # obj <- SetIdent(obj, value = metadata_column)

  total_counts <- table(seurat_obj@meta.data[[metadata_column]])

  low_count_cells <- names(total_counts[total_counts < threshold])

  seurat_obj <- subset(seurat_obj, idents = low_count_cells, invert = TRUE)

  return(seurat_obj)
}
test <- remove_low_count_cells(seurat_obj = test, metadata_column = "blueprint.main", threshold = 20)

In [ ]:
table(test@meta.data$blueprint.main)

In [ ]:
# test <- SetIdent(test, value = "blueprint.main")
# test <- subset(test, idents = c("Adipocytes", "Erythrocytes", "HSC"), invert = TRUE)
# table(test@meta.data$blueprint.main)

In [ ]:
sce <- LayerData(test)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
test@meta.data$monaco.fine <- monaco.fine$pruned.labels
save(test, file=file.path('objects', '05.rda'))

In [ ]:
# load(file = file.path('objects', '05.rda'))

In [ ]:
table(test@meta.data$monaco.fine)

In [ ]:
test <- SetIdent(test, value = "monaco.fine")
remove_low_count_cells <- function(seurat_obj, metadata_column, threshold = 20) {

  # obj <- SetIdent(obj, value = metadata_column)

  total_counts <- table(seurat_obj@meta.data[[metadata_column]])

  low_count_cells <- names(total_counts[total_counts < threshold])

  seurat_obj <- subset(seurat_obj, idents = low_count_cells, invert = TRUE)

  return(seurat_obj)
}
test <- remove_low_count_cells(seurat_obj = test, metadata_column = "monaco.fine", threshold = 20)

In [ ]:
table(test@meta.data$monaco.fine)

In [ ]:
# Genes Rik was interested
genes <- 
c("LILRB1", "LILRB2", "LILRB3", "HLA-G", "IL1B",
 "IL6", "IL23A", "IL12A", "IL12B", "TNF", "IL10", "TIGIT", "PDCD1")

In [ ]:
# Keep only genes present in data for `default.multigene`
seurat_data <- GetAssayData(object = test)
seurat_genes <- rownames(seurat_data)
present_genes <- genes[genes %in% seurat_genes]

In [ ]:
colnames(test@meta.data)

In [ ]:

# Clean up metadata
test@meta.data$Gender <- test@meta.data$gender.ch1
test@meta.data$Stim <- test@meta.data$disease.state.ch1
test@meta.data$Orig.Annotations <- test@meta.data$annotation

In [ ]:
columns_to_keep <-  c('orig.ident', 'nCount_RNA', 'Stim', 'Gender', 'Orig.Annotations', 'blueprint.main', 'monaco.fine')
test@meta.data <- test@meta.data[, columns_to_keep, drop = FALSE]

In [ ]:
test <- subset(test, subset = Orig.Annotations != "NA")

In [ ]:
save(test, file=file.path('objects', '06.rda'))

In [ ]:

seu =  test
scConf = createConfig(seu)
makeShinyApp(seu, scConf,
             gene.mapping = TRUE,
             shiny.title = "UC and Crohn's IBD from colonic mucosa scRNAseq",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 
system("R -e \"shiny::runApp('colon_UC_crohns_scRNA-seq_GSM6614348')\"")

In [ ]:
# All stims
seu =  test
scConf1 = createConfig(seu)
makeShinyFiles(seu, scConf1,
             gene.mapping = TRUE,
             shiny.prefix = "sc1",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 

In [ ]:
# Ulcerative colitis
load(file=file.path('objects', '06.rda'))
test <- subset(test, subset = Stim == "ulcerative colitis")
test <- RunPCA(test, npcs = 30, verbose = TRUE)
test <- RunUMAP(test, reduction = "pca", dims = 1:20)
test <- FindNeighbors(test, reduction = "pca", dims = 1:20)
test <- FindClusters(test, resolution = 0.5)

save(test, file=file.path('objects', 'uc.rda'))

seu =  test
scConf2 = createConfig(seu)
makeShinyFiles(seu, scConf2,
             gene.mapping = TRUE,
             shiny.prefix = "sc2",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 

In [ ]:
# Crohn's disease
load(file=file.path('objects', '06.rda'))
test <- subset(test, subset = Stim == "Crohn’s disease")
test <- RunPCA(test, npcs = 30, verbose = TRUE)
test <- RunUMAP(test, reduction = "pca", dims = 1:20)
test <- FindNeighbors(test, reduction = "pca", dims = 1:20)
test <- FindClusters(test, resolution = 0.5)

save(test, file=file.path('objects', 'cd.rda'))

seu =  test
scConf3 = createConfig(seu)
makeShinyFiles(seu, scConf3,
             gene.mapping = TRUE,
             shiny.prefix = "sc3",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 

In [ ]:
# Controls
load(file=file.path('objects', '06.rda'))
test <- subset(test, subset = Stim == "healthy control")
test <- RunPCA(test, npcs = 30, verbose = TRUE)
test <- RunUMAP(test, reduction = "pca", dims = 1:20)
test <- FindNeighbors(test, reduction = "pca", dims = 1:20)
test <- FindClusters(test, resolution = 0.5)

save(test, file=file.path('objects', 'hc.rda'))

seu =  test
scConf4 = createConfig(seu)
makeShinyFiles(seu, scConf4,
             gene.mapping = TRUE,
             shiny.prefix = "sc4",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 

In [ ]:
makeShinyCodesMulti(
  shiny.title = "UC and Crohn's IBD from colonic mucosa scRNAseq", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2", "sc3", "sc4"),
  shiny.headers = c("All", "UC", "CD", "Healthy"), 
  shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/") 

In [ ]:
system("R -e \"shiny::runApp('colon_UC_crohns_scRNA-seq_GSM6614348')\"")